# Multi-Json

In [9]:
import json
import csv
import os
import argparse

In [12]:
results_dir = "./results"

# Reads all of the json files in the dir
def list_json_files(directory):
    json_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))
    return json_files

list_dir = list_json_files(results_dir)


In [36]:
def above_two(data_records):
    index=[]
    for i in range(len(data_records)):
        modules = data_records[i]["modules"].keys()
        if len(modules) > 2:
            # print(i, len(modules))
            index.append(i)
        else:
            continue
    return index

def process_files(file_paths):
    
    csv_filename = 'comb_output.csv'
    
    # Check if the output file already exists
    file_exists = os.path.isfile(csv_filename)
    
    with open(csv_filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        if not file_exists:
            writer.writerow(["Bin_ID", "Contig_ID", "Gene_Coordinates", "Ctg_Number", "Ctg_Tag_Id", "Mibig_biosynthetic_gene_cluster", "Mibig_Product", "Blast_name", "Blast_genecluster","Blast_annotation","Blast_perc_coverage", "Blast_perc_ident", "Blast_blastscore", "Blast_evalue", "Blast_locus_tag" ])
        
        for file_path in file_paths:

            with open(file_path, 'r') as files:
                data = json.load(files)
                index_ = above_two(data["records"])
                
                for i in index_:
                    bin_id = data["input_file"]
                    contig_id = data["records"][i]["modules"]["antismash.modules.clusterblast"]["record_id"]
                    blast_info = data["records"][i]["modules"]["antismash.modules.clusterblast"]["general"]["results"][0]["ranking"][1][1]["pairings"]
                    mibig_entries = data["records"][i]["modules"]["antismash.modules.clusterblast"]["knowncluster"]["mibig_entries"]["1"]

                    # Dict for each ctg_tagid
                    blast_ = {}
                    for blast in blast_info:
                        ctg_name = blast[0].split("|")[4]
                        # ctg_cordinates = blast[0].split("|")[2]
                        blast_["ctg_cordinates"] = blast[0].split("|")[2]
                        blast_name, blast_genecluster, blast_start, blast_end, blast_strand, blast_annotation, blast_perc_ident, blast_blastscore, blast_perc_coverage, blast_evalue, blast_locus_tag  = blast[2].values()
                        blast_[ctg_name] = {"blast_name" : blast_name, 
                                            "blast_genecluster": blast_genecluster,
                                            "blast_annotation": blast_annotation,
                                            "blast_perc_coverage":blast_perc_coverage,
                                            "blast_perc_ident": blast_perc_ident,
                                            "blast_blastscore": blast_blastscore,
                                            "blast_evalue": blast_evalue,
                                            "blast_locus_tag": blast_locus_tag,
                                            "cordinates": blast[0].split("|")[2] }

                    for ctg_tagid, vl in mibig_entries.items():
                        ctg_id = ctg_tagid.split("_")[0]
                        mibig_entry = vl[0]
                        biosynthetic_gene_cluster_id = mibig_entry[2]
                        mibig_entries_product = mibig_entry[4]
                        blast_info_ = blast_.get(ctg_tagid)

                        if blast_info_:
                            ctg_cordinates = blast_info_["cordinates"]
                            blast_name = blast_info_["blast_name"]
                            blast_genecluster= blast_info_["blast_genecluster"]
                            blast_annotation = blast_info_["blast_annotation"]
                            blast_perc_coverage = blast_info_["blast_perc_coverage"]
                            blast_perc_ident = blast_info_["blast_perc_ident"]
                            blast_blastscore = blast_info_["blast_blastscore"]
                            blast_evalue = blast_info_["blast_evalue"]
                            blast_locus_tag = blast_info_["blast_locus_tag"]

                        else:
                            ctg_cordinates = None
                            blast_name = None
                            blast_genecluster= None
                            blast_annotation = None
                            blast_perc_coverage = None
                            blast_perc_ident = None
                            blast_blastscore = None
                            blast_evalue = None
                            blast_locus_tag = None
                            
                        writer.writerow([
                            bin_id, 
                            contig_id,
                            ctg_cordinates, 
                            ctg_id, 
                            ctg_tagid, 
                            biosynthetic_gene_cluster_id, 
                            mibig_entries_product, 
                            blast_name,
                            blast_genecluster,
                            blast_annotation,
                            blast_perc_coverage,
                            blast_perc_ident,
                            blast_blastscore,
                            blast_evalue,
                            blast_locus_tag
                        ])
            

process_files(list_dir)

In [38]:
import pandas as pd
pd.read_csv("./comb_output.csv")

,Bin_ID,Contig_ID,Gene_Coordinates,Ctg_Number,Ctg_Tag_Id,Mibig_biosynthetic_gene_cluster,Mibig_Product,Blast_name,Blast_genecluster,Blast_annotation,Blast_perc_coverage,Blast_perc_ident,Blast_blastscore,Blast_evalue,Blast_locus_tag
0,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,NaN,ctg3,ctg3_27,BGC0000588,RiPP:Microcin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,28598-29438,ctg3,ctg3_28,BGC0000607,RiPP:Thiopeptide,CG392_RS04930,NZ_NNRS01000004_c78906-120244,AAA_family_ATPase,99.641577,100.0,536.0,1.450000e-193,WP_004121099.1
2,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,43941-44757,ctg3,ctg3_38,BGC0002284,Other,CG392_RS04985,NZ_NNRS01000004_c78906-120244,(p)ppGpp_synthetase,100.000000,95.0,493.0,8.280000e-177,WP_116438271.1
3,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,47230-48667,ctg3,ctg3_42,BGC0002561,Alkaloid,CG392_RS05005,NZ_NNRS01000004_c78906-120244,hydroxymethylglutaryl-CoA_synthase,101.673640,84.0,765.0,4.340000e-278,WP_116438274.1
4,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,50447-51650,ctg3,ctg3_44,BGC0000867,Other,CG392_RS05015,NZ_NNRS01000004_c78906-120244,acetyl-CoA_C-acyltransferase,106.500000,91.0,727.0,9.470000e-265,WP_116438276.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,MG_323829.bin1.fa,NODE_125_length_11688_cov_13.230809,11096-11687,ctg47,ctg47_16,BGC0000948,Other,DX05_RS06925,NZ_CP007572_c1349890-1391168,diacylglycerol_kinase_family_lipid_kinase,100.000000,100.0,380.0,6.470000e-133,WP_000833762.1
146,MG_323757.bin1.fa,NODE_83_length_8923_cov_7.096640,NaN,ctg59,ctg59_6,BGC0000100,Polyketide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
147,MG_323757.bin1.fa,NODE_110_length_7016_cov_6.405832,520-1279,ctg82,ctg82_1,BGC0002667,RiPP,A5816_RS11485,NZ_NGLI01000002_c555810-572363,LytTR_family_DNA-binding_domain-containing_pro...,98.809524,93.0,460.0,1.940000e-164,WP_010733065.1
148,MG_323757.bin1.fa,NODE_110_length_7016_cov_6.405832,1280-2594,ctg82,ctg82_2,BGC0000624,RiPP,A5816_RS11480,NZ_NGLI01000002_c555810-572363,GHKL_domain-containing_protein,72.311213,88.0,527.0,6.040000e-187,WP_080114483.1


# Going Modular

In [20]:
%%writefile python_scripts/json_csv.py 
import json
import csv
import os
import argparse

# Reads all of the json files in the dir
def list_json_files(directory):
    json_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))
    return json_files

def above_two(data_records):
    index=[]
    for i in range(len(data_records)):
        modules = data_records[i]["modules"].keys()
        if len(modules) > 2:
            # print(i, len(modules))
            index.append(i)
        else:
            continue
    return index

def process_files(file_paths, csv_filename):
    
    # Check if the output file already exists
    file_exists = os.path.isfile(csv_filename)
    
    with open(csv_filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        if not file_exists:
            writer.writerow(["Bin_ID", "Contig_ID", "Gene_Coordinates", "Ctg_Number", "Ctg_Tag_Id", "Mibig_biosynthetic_gene_cluster", "Mibig_Product", "Blast_name", "Blast_genecluster","Blast_annotation","Blast_perc_coverage", "Blast_perc_ident", "Blast_blastscore", "Blast_evalue", "Blast_locus_tag" ])
        
        for file_path in file_paths:
            with open(file_path, 'r') as files:
                data = json.load(files)
                index_ = above_two(data["records"])

                for i in index_:
                    bin_id = data["input_file"]
                    contig_id = data["records"][i]["modules"]["antismash.modules.clusterblast"]["record_id"]
                    blast_info = data["records"][i]["modules"]["antismash.modules.clusterblast"]["general"]["results"][0]["ranking"][1][1]["pairings"]
                    mibig_entries = data["records"][i]["modules"]["antismash.modules.clusterblast"]["knowncluster"]["mibig_entries"]["1"]

                    # Dict for each ctg_tagid
                    blast_ = {}
                    for blast in blast_info:
                        ctg_name = blast[0].split("|")[4]
                        # ctg_cordinates = blast[0].split("|")[2]
                        blast_["ctg_cordinates"] = blast[0].split("|")[2]
                        blast_name, blast_genecluster, blast_start, blast_end, blast_strand, blast_annotation, blast_perc_ident, blast_blastscore, blast_perc_coverage, blast_evalue, blast_locus_tag  = blast[2].values()
                        blast_[ctg_name] = {"blast_name" : blast_name, 
                                            "blast_genecluster": blast_genecluster,
                                            "blast_annotation": blast_annotation,
                                            "blast_perc_coverage":blast_perc_coverage,
                                            "blast_perc_ident": blast_perc_ident,
                                            "blast_blastscore": blast_blastscore,
                                            "blast_evalue": blast_evalue,
                                            "blast_locus_tag": blast_locus_tag,
                                            "cordinates": blast[0].split("|")[2] }

                    for ctg_tagid, vl in mibig_entries.items():
                        ctg_id = ctg_tagid.split("_")[0]
                        mibig_entry = vl[0]
                        biosynthetic_gene_cluster_id = mibig_entry[2]
                        mibig_entries_product = mibig_entry[4]
                        blast_info_ = blast_.get(ctg_tagid)

                        if blast_info_:
                            ctg_cordinates = blast_info_["cordinates"]
                            blast_name = blast_info_["blast_name"]
                            blast_genecluster= blast_info_["blast_genecluster"]
                            blast_annotation = blast_info_["blast_annotation"]
                            blast_perc_coverage = blast_info_["blast_perc_coverage"]
                            blast_perc_ident = blast_info_["blast_perc_ident"]
                            blast_blastscore = blast_info_["blast_blastscore"]
                            blast_evalue = blast_info_["blast_evalue"]
                            blast_locus_tag = blast_info_["blast_locus_tag"]

                        else:
                            ctg_cordinates = None
                            blast_name = None
                            blast_genecluster= None
                            blast_annotation = None
                            blast_perc_coverage = None
                            blast_perc_ident = None
                            blast_blastscore = None
                            blast_evalue = None
                            blast_locus_tag = None
                            
                        writer.writerow([
                            bin_id, 
                            contig_id,
                            ctg_cordinates, 
                            ctg_id, 
                            ctg_tagid, 
                            biosynthetic_gene_cluster_id, 
                            mibig_entries_product, 
                            blast_name,
                            blast_genecluster,
                            blast_annotation,
                            blast_perc_coverage,
                            blast_perc_ident,
                            blast_blastscore,
                            blast_evalue,
                            blast_locus_tag
                        ])
            
def main():
    parser = argparse.ArgumentParser(description="Process JSON files and output combined results to a CSV file. Please note dont run with the same output name file it will add the same results in the file.")
    parser.add_argument('--result_dir',"-r", help='JSON files to process')
    parser.add_argument('-o', '--output', default='combined_output.csv', help='Output CSV file name (default: combined_output.csv)')
    
    args = parser.parse_args()
    
    list_jsons = list_json_files(args.result_dir)

    process_files(list_jsons, args.output)
    print(f"Output written to: {args.output}")

if __name__ == "__main__":
    main()

Overwriting python_scripts/json_csv.py


In [21]:
!python3 python_scripts/json_csv.py -r "./results" -o "test_going_modular.csv"

Output written to: test_going_modular.csv


In [39]:
!python3 python_scripts/json_csv.py --help

usage: json_csv.py [-h] [--result_dir RESULT_DIR] [-o OUTPUT]

Process JSON files and output combined results to a CSV file.

options:
  -h, --help            show this help message and exit
  --result_dir RESULT_DIR, -r RESULT_DIR
                        JSON files to process
  -o OUTPUT, --output OUTPUT
                        Output CSV file name (default: combined_output.csv)


In [23]:
import pandas as pd
pd.read_csv("./test_going_modular.csv")

,Bin_ID,Contig_ID,Gene_Coordinates,Ctg_Number,Ctg_Tag_Id,Mibig_biosynthetic_gene_cluster,Mibig_Product,Blast_name,Blast_genecluster,Blast_annotation,Blast_perc_coverage,Blast_perc_ident,Blast_blastscore,Blast_evalue,Blast_locus_tag
0,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,NaN,ctg3,ctg3_27,BGC0000588,RiPP:Microcin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,28598-29438,ctg3,ctg3_28,BGC0000607,RiPP:Thiopeptide,CG392_RS04930,NZ_NNRS01000004_c78906-120244,AAA_family_ATPase,99.641577,100.0,536.0,1.450000e-193,WP_004121099.1
2,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,43941-44757,ctg3,ctg3_38,BGC0002284,Other,CG392_RS04985,NZ_NNRS01000004_c78906-120244,(p)ppGpp_synthetase,100.000000,95.0,493.0,8.280000e-177,WP_116438271.1
3,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,47230-48667,ctg3,ctg3_42,BGC0002561,Alkaloid,CG392_RS05005,NZ_NNRS01000004_c78906-120244,hydroxymethylglutaryl-CoA_synthase,101.673640,84.0,765.0,4.340000e-278,WP_116438274.1
4,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,50447-51650,ctg3,ctg3_44,BGC0000867,Other,CG392_RS05015,NZ_NNRS01000004_c78906-120244,acetyl-CoA_C-acyltransferase,106.500000,91.0,727.0,9.470000e-265,WP_116438276.1
5,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,57963-58755,ctg3,ctg3_50,BGC0000291,NRP,CG392_RS05045,NZ_NNRS01000004_c78906-120244,ABC_transporter_ATP-binding_protein,97.718631,89.0,429.0,7.930000e-152,WP_162862662.1
6,MG_323829.bin2.fa,NODE_3_length_155941_cov_99.724542,60451-62272,ctg3,ctg3_52,BGC0001038,NRP+Polyketide:Modular type I polyketide,CG392_RS05055,NZ_NNRS01000004_c78906-120244,galactose-1-phosphate_uridylyltransferase,100.000000,98.0,1181.0,0.000000e+00,WP_116438279.1
7,MG_323757.bin3.fa,NODE_63_length_11229_cov_186.926347,2192-3056,ctg15,ctg15_4,BGC0000530,RiPP:Lanthipeptide,G6Z86_RS06415,NZ_CP049223_c1306777-1329486,Rgg/GadR/MutR_family_transcriptional_regulator,100.000000,100.0,552.0,1.070000e-199,WP_006731082.1
8,MG_323757.bin3.fa,NODE_63_length_11229_cov_186.926347,3369-4116,ctg15,ctg15_5,BGC0002122,NRP,G6Z86_RS06420,NZ_CP049223_c1306777-1329486,ABC_transporter_permease,100.000000,100.0,488.0,1.060000e-175,WP_006732653.1
9,MG_323757.bin3.fa,NODE_63_length_11229_cov_186.926347,4108-4849,ctg15,ctg15_6,BGC0000539,RiPP:Lanthipeptide,G6Z86_RS06425,NZ_CP049223_c1306777-1329486,ABC_transporter_permease,100.000000,100.0,458.0,4.980000e-164,WP_006731087.1
